In [1]:
import random
from random import randrange

QBER=0.0
asja_key=[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]
balvis_key=[0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1]


def split(list1, n): 
    out = []
    last = 0.0
    while last < len(list1):
        out.append(list1[int(last):int(last + n)])
        last += n
    return out

def cascade_pass(lA, lB, n): #input key lists A-Asja, B-Balvis and target block size to divide in blocks
    #Shuffle
    permutation = list(zip(lA, lB)) #map the index of multiple lists
    random.shuffle(permutation) #performing permutation
    shuffledLA, shuffledLB = zip(*permutation) #unpacking values
    #Split
    splitLA=split(shuffledLA, n)
    splitLB=split(shuffledLB, n)
    #Calculate parity
    #Creating empty lists, were "correctA/B" will include blocks with no error found
    #And "errorA/B" - list with blocks where parities dismatched
    correctA, correctB, errorA, errorB= [], [], [], []
    sumBlocksA = [sum(block) for block in splitLA] # calculating parity by first calculating sums of each block in splitA/B
    sumBlocksB = [sum(block) for block in splitLB]
    parityA = [i %2 for i in sumBlocksA] #then aplying mod(2) operator to our calculated sums and saving results
    parityB = [i %2 for i in sumBlocksB] #in parity bit list
    for i,value in enumerate(range(len(parityA))): #comparing parities from list1 with list2
        if parityA[i]==parityB[i]: #if parities mathced - we add corresponding blocks to our list 'correct'
            correctA.append(splitLA[i])
            correctB.append(splitLB[i])
        else:
            errorA.append(splitLA[i]) #if parities dismatched - we add corresponding blocks to our list 'errors'
            errorB.append(splitLB[i])
    keyA = [item for i in correctA for item in i] #Converting our correct blocks into a list
    keyB= [item for i in correctB for item in i]
    return keyA, keyB, errorA, errorB #returning key that consist of correct blocks (list) and blocks with errors (tuple)

#Solution
#Information reconciliation
#Before starting error correction, we check calculated QBER value
print(QBER)
if QBER==0.0:
    print("QBER is 0")
    print("Final Key Asja", asja_key)
    print("Final Key Balvis", balvis_key)
if QBER>=0.25: 
    print("QBER value is", QBER,"\nThreshold value reached!") #If QBER is above threshold value - we abort protocol
if 0<QBER<=0.25: #if 0<QBER<=0.25 we perform Cascade protocol
    blockSize=0.73//QBER
    kFinalA, kFinalB=[], [] #creating registers for final keys
    #Cascade protocol 1st pass
    corrBlockA, corrBlockB, errBlockA, errBlockB=cascade_pass(asja_key, balvis_key, blockSize) #cascade function
    kFinalA.extend(corrBlockA) #adding block which parities matched to final key string
    kFinalB.extend(corrBlockB)
    print("---RESULTS AFTER 1ST PASS---\n", "Final key Asja:", kFinalA, "\n", "Final key Balvis", kFinalB)
    print(" Blocks with errors Asja", errBlockA, "\n", "Blocks with errors Balvis", errBlockB)
    
    #We now aproximately know how many errors we have in intial key string,
    #because after first pass each block in errorA/B list contains 1 (or other odd number) of errors
    #We now can determine the final (corrected) key list lenght before we correct that errors (when 1 bit left in each block)
    #In other words, a key lenght we will have in penultimate pass of our Cascade protocol
    n=0
    penultimatePassLenght=len(asja_key)-len(errBlockA)
    while len(kFinalA)!=penultimatePassLenght: #Bisective search at each block until corrected key lenght is not equal lenght of initial key minus error blocks number after first pass
        for i, (blockA, blockB) in enumerate(zip(errBlockA, errBlockB)): #We will work with each block that contains and error
            if len(blockA)>1:
                secondPassA=list(blockA)# we convert block into a lists
                secondPassB=list(blockB)
                blockSize2=len(blockA)//2 #we change block size, now we will divide each block that contains an error in halfs
                print(blockSize2)
                corrBlockA2, corrBlockB2,  errBlockA2, errBlockB2=cascade_pass(secondPassA, secondPassB, blockSize2) #and apply cascade
                kFinalA.extend(corrBlockA2) #then we will add correct bits to key strings
                kFinalB.extend(corrBlockB2)
                print(corrBLockA2)
                errBlockA[i]=errBlockA2 #updating error block values
                errBlockB[i]=errBlockB2
                print(errBlockA2)
            if len(blockA)==1: #Side case if one block in the round will be shorter than the oner thus will require less passes
                for bit in blockA:
                    if bit==1:
                        bitA=errBlockA[0][0]
                        kFinalA.append(bitA)#Asja adds corresponding bit to her key string without change
                        bitB=errBlockB[0][0]+1 #but Balvis will first correct the error by flipping the bit value 
                        kFinalB.append(bitB)
                    if bit==0:
                        bitA=errBlockA[0][0]
                        kFinalA.append(bitA) #Asja adds corresponding bit to her key string without change
                        bitB=errBlockB[0][0]-1 #but Balvis will first correct the error by flipping the bit value 
                        kFinalB.append(bitB)
                        
    print("---PERFORMING NEXT PASS---\n", "Final key Asja:", kFinalA, "\n", "Final key Balvis", kFinalB)
    print(" Blocks with errors Asja", errBlockA, "\n", "Blocks with errors Balvis", errBlockB)
    
    #After Second pass we have a nested lists, to convert them:    
    errorA=[item for elem in errBlockA for item in elem]
    errorB=[item for elem in errBlockB for item in elem]
    
    #Error correction step, when our error blocks contains just 1 bit (error)
    for i, error in enumerate(zip(errorA, errorB)):
        bitA=int(''.join(map(str, errorA[i]))) #Converting tuple to integer
        bitB=int(''.join(map(str, errorB[i])))
        if bitA==1:
            kFinalA.append(bitA)
            correctedBitB=bitB+1
            kFinalB.append(correctedBitB)
        if bitA==0:
            kFinalA.append(bitA)
            correctedBitB=bitB-1
            kFinalB.append(correctedBitB)
            
    print("Final Key Asja", kFinalA)
    print("Final Key Balvis", kFinalB)

0.0
QBER is 0
Final Key Asja [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]
Final Key Balvis [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1]


In [2]:
from numpy import log as ln

if QBER!=0: #defining size of blocks
    biconfBlockSize=(4*ln(2))//(3*QBER)
if QBER==0:
    biconfBlockSize=8
    kFinalA=asja_key
    kFinalB=balvis_key
#print(QBER)

rounds = 0 #counting rounds
biconfError=[] #creating register for rounds with an error
error=0 #register for found and corrected error

while rounds!=8: #we will go through rounds and monitor if blocks with errors will be found 
    rounds=rounds+1
    #Creating random subsets
    kFinalZipped=list(zip(kFinalA, kFinalB)) #maping indexes of our two lists
    randomBlock=random.sample(list(enumerate(kFinalZipped)), int(biconfBlockSize))
    #at this point we will have nested tuple that contains (index of random bit, (bit from Asja string, bit from Balvis string))
    #print(randomBitList) #will print out our nested tuple
    #print(randomBitList[0]) # will print out one block (index, (bitA, bitB))
    #print(randomBitList[0][0]) # will print only first pair index
    #print(randomBitList[0][1][0]) #will print only first pair Asjas' bit
    
    #We will now need to calculate and compare parity bits for both users bits
    sumBlockA=0
    sumBlockB=0
    for i in range(0,int(biconfBlockSize)):
        sumBlockA=sumBlockA+randomBlock[i][1][0]
        sumBlockB=sumBlockB+randomBlock[i][1][1]
    parityA = sumBlockA%2 #then aplying mod(2) operator to our calculated sums and saving results
    parityB = sumBlockB%2
    
    if parityA!=parityB: #if parities of block dismatch - we bisective search to correct error before continue with next round
        print("Error found in round:", rounds)
        print("Applying bisective search and error correction")
        #Applying bisective search to find and correct an error
        while len(randomBlock)>1: #We will take our block with error and run besective search till we find bit with error
            half=len(randomBlock)//2 #Determining the lenght of each half
            splitBlock=split(randomBlock, half) #spliting our block in two parts
            for i, block in enumerate(splitBlock): #For each part
                sumA=0
                sumB=0
                for j in range(0,len(splitBlock[0])): #we will calculate sums 
                    sumA=sumA+splitBlock[i][j][1][0]
                    sumB=sumB+splitBlock[i][j][1][1]
                parA=sumA%2 #then calculate parities
                parB=sumB%2
                if parA!=parB: #if parities dismatch- we update our block and run while loop again
                    randomBlock=splitBlock[i]
        if len(randomBlock)==1: #once we isolate the error to 1 bit
            error=error+1
            print("Error found in bit:", randomBlock[0][0]) #we retrieving the index of bit pair
            errorIndex=int(randomBlock[0][0])
            #Apply error correction at Balvis' initial key string
        if kFinalB[errorIndex]==0:
            kFinalB[errorIndex]=1
        else:
            kFinalB[errorIndex]=0
        print("Error corrected!\n")
    else: #If parities matched
        pass

print("BICONF strategy completed!\n", error, "errors found!")
print("Final key Asja", kFinalA)
print("Final key Balvis", kFinalB)

Error found in round: 1
Applying bisective search and error correction
Error found in bit: 2
Error corrected!

Error found in round: 2
Applying bisective search and error correction
Error found in bit: 10
Error corrected!

Error found in round: 3
Applying bisective search and error correction
Error found in bit: 7
Error corrected!

BICONF strategy completed!
 3 errors found!
Final key Asja [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]
Final key Balvis [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]
